In [1]:
!pip install mlxtend

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 6.9 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.6/248.6 kB 3.5 MB/s eta 0:00:00a 0:00:01
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 5.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 1.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
import json 
from mlxtend.frequent_patterns import apriori, association_rules

In [4]:
df = pd.read_csv("client_services.csv")
df = df[df.columns[1:]]
df

,Serv_1,Serv_2,Serv_3,Serv_4,Serv_5,Serv_6,Serv_7,Serv_8,Serv_9,Serv_10,Serv_11,Serv_12,Serv_13,Serv_14,Serv_15,Serv_16,Serv_17,Serv_18,Serv_19,Serv_20,Serv_21,Serv_22,Serv_23,Serv_24,Serv_25,Serv_26,Serv_27,Serv_28,Serv_29,Serv_30,Serv_31,Serv_32,Serv_33,Serv_34,Serv_35,Serv_36,Serv_37,Serv_38,Serv_39,Serv_40,...,Serv_61,Serv_62,Serv_63,Serv_64,Serv_65,Serv_66,Serv_67,Serv_68,Serv_69,Serv_70,Serv_71,Serv_72,Serv_73,Serv_74,Serv_75,Serv_76,Serv_77,Serv_78,Serv_79,Serv_80,Serv_81,Serv_82,Serv_83,Serv_84,Serv_85,Serv_86,Serv_87,Serv_88,Serv_89,Serv_90,Serv_91,Serv_92,Serv_93,Serv_94,Serv_95,Serv_96,Serv_97,Serv_98,Serv_99,Serv_100
0,0,0,0,1,1,0,1,0,1,1,0,1,1,0,1,0,1,1,1,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,1,1,0,1,0,1,...,0,0,1,1,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,1,0,0,1,1,0,1,1,0,0,1,0,0,1,1,1,0,1,1,0,0
1,1,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,1,0,0,1,1,0,1,1,0,0,0,1,1,1,1,0,1,0,1,1,1,1,1,1,...,1,1,1,0,0,0,1,1,0,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,1,1,0,0,0,0,1
2,1,0,1,0,0,1,0,0,0,0,1,1,0,1,0,1,1,0,1,0,0,0,0,0,1,1,0,1,1,0,0,0,1,1,0,0,1,0,0,1,...,0,0,0,1,0,0,1,0,0,1,1,1,1,1,1,0,1,1,1,0,1,1,1,0,1,1,0,0,1,1,1,0,1,1,0,0,1,0,1,0
3,0,1,0,1,0,1,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,1,0,1,0,1,0,0,1,0,...,0,1,0,0,1,1,0,1,1,0,1,1,1,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,1,0,1,1,1,0,0,0
4,1,1,0,1,1,1,0,1,1,0,0,0,0,1,1,0,1,0,0,1,1,0,1,0,0,0,1,1,1,1,0,1,1,1,1,0,0,1,1,1,...,0,0,1,1,1,1,0,1,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,1,1,1,1,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,1,1,0,0,1,0,0,1,0,0,0,1,0,1,1,0,1,1,0,1,1,1,0,1,0,1,0,0,0,0,0,1,0,0,1,0,1,1,0,...,0,1,0,1,1,0,1,1,0,1,1,1,0,1,1,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,1,0,1,1,0,0
96,0,1,0,1,1,0,0,0,0,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,1,1,...,1,1,0,1,0,1,0,1,0,0,1,0,0,0,1,0,0,1,1,0,1,0,1,0,0,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0
97,1,0,0,1,1,0,1,0,0,0,0,1,1,0,0,1,1,0,1,1,0,1,1,1,0,1,1,0,0,0,1,1,1,0,1,0,0,0,0,1,...,1,1,0,1,1,1,0,1,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,1,0,1,1,0,0,1
98,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,0,1,1,1,1,1,0,0,0,0,1,1,1,0,0,...,1,1,1,0,0,1,0,0,1,1,1,1,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,1,1,1,1,0,1,0,0,0,0,0


In [5]:
support_values = apriori(df, min_support=0.2, use_colnames=True)
support_values.sort_values(['support'], ascending=False, inplace=True)
rules = association_rules(support_values, metric='confidence', min_threshold=0.4)
rules.sort_values(['confidence', 'lift'], ascending=[False, False], inplace=True)

In [6]:
def get_antecedents_exact_match(services, existing_services):
    diff = services.difference(existing_services)
    if len(diff) == 0:
        return True
    else:
        return False

In [7]:
def get_antecedents_partial_match(services, existing_services):
    intersect = services.intersection(existing_services)
    if len(intersect) > 0:
        return True
    else:
        return False

In [24]:
def generate_recommendations(existing_services, num_recommendations):
    recommend_dictionary = dict()
    frozen_existing_services = frozenset(existing_services)
    recommendations = rules[rules.apply(lambda rule: get_antecedents_exact_match(rule['antecedents'], frozen_existing_services), axis=1)]
    if len(recommendations) == 0:
        recommendations = rules[rules.apply(lambda rule: get_antecedents_partial_match(rule['antecedents'], frozen_existing_services), axis=1)]
    if len(recommendations) == 0:
        print('No recommendations available!')
        return recommend_dictionary
    
    all_recommendations = recommendations.sort_values(['confidence', 'lift'], ascending=[False, False])

    num_available_recommendations = len(all_recommendations)
    num_sys_provided_recommendations = min(num_recommendations, num_available_recommendations)

    if num_available_recommendations > 0:
        consequents = recommendations['consequents'].values
        confidences = recommendations['confidence'].values
        
        for i in range(num_sys_provided_recommendations):
            temp_dict= dict()
            temp_dict['services'] = list(consequents[i])
            temp_dict['confidence'] = round(confidences[i], 2)*100
            recommend_dictionary['recommendations_'+str(i)] = temp_dict
        recommend_json = json.dumps(recommend_dictionary, indent = 4) 
    return recommend_json

In [30]:
EXISTING_SERVICES = ['Serv_80', 'Serv_65']
#EXISTING_SERVICES = ['new_service']
NUM_RECOMMENDATIONS = 2
print(generate_recommendations(EXISTING_SERVICES, NUM_RECOMMENDATIONS))

{
    "recommendations_0": {
        "services": [
            "Serv_32"
        ],
        "confidence": 91.0
    },
    "recommendations_1": {
        "services": [
            "Serv_32"
        ],
        "confidence": 71.0
    }
}
